# Principal Component Analysis (PCA)

Principle Component Analysis (PCA) is a standard tool in modern data analysis. PCA Provides a roadmap for how to reduce a complex data set to a lower dimension to reveal the sometimes hidden, simplified structures that often underline it. The goal of this posting is to provide some intuitive understanding of PCA via some coding examples. To understand what's happening behind the scene (mathematics) you need to have a sound knowledge of linear algebra (matrix algebra).

# Reading

There are plenty of literature out there on PCA, among all I found following paper and the video is well written and explain all the nuance in PCA.

Paper: https://arxiv.org/pdf/1404.1100.pdf

Lecture: [video](https://www.youtube.com/watch?v=a9jdQGybYmE "link title")

In [2]:
import pandas as pd
import numpy as np
import numpy as np
from sklearn.decomposition import PCA
from IPython.display import display, HTML
from sympy import init_printing, Matrix, symbols, sqrt
init_printing(use_latex = 'mathjax')

# Naive Basis

Each sample (row) $\vec{X}$ is a $m=9$ dimentional vector, each dimention is a some measurement of the location. There are 329 vectors in the dataset. Since each vector is a $m$ dimentional, the vector lies in an $m$-dimentional vector space span by some orthonormal basis. Lets say the basis are $[b_1...b_9]^T$

Now we can write a vector $\vec{X} =b_1x_1 + b_2x_2 + b_3x_3 + b_4x_4 + b_5x_5 + b_6x_6 + b_7x_7 + b_8x_8 + b_9x_9$


we can write the basis as $m\times m$ metrix $B$. We could use any orthonormal basis, but the naive choice would be identity matrix. $B$ is call the **naive basis** (naive basis reflects the method we gathered the data) 


In [4]:
B = Matrix([[1, 0, 0, 0, 0, 0, 0, 0, 0],[0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0],[0,0, 0, 1, 0, 0, 0, 0, 0],[0,0, 0, 0, 1, 0, 0, 0, 0],[0,0, 0, 0, 0, 1, 0, 0, 0],[0,0, 0, 0, 0, 0, 1, 0, 0],[0,0, 0, 0, 0, 0, 0, 1, 0],[0,0, 0, 0, 0, 0, 0, 0, 1]])
display(B)

⎡1  0  0  0  0  0  0  0  0⎤
⎢                         ⎥
⎢0  1  0  0  0  0  0  0  0⎥
⎢                         ⎥
⎢0  0  1  0  0  0  0  0  0⎥
⎢                         ⎥
⎢0  0  0  1  0  0  0  0  0⎥
⎢                         ⎥
⎢0  0  0  0  1  0  0  0  0⎥
⎢                         ⎥
⎢0  0  0  0  0  1  0  0  0⎥
⎢                         ⎥
⎢0  0  0  0  0  0  1  0  0⎥
⎢                         ⎥
⎢0  0  0  0  0  0  0  1  0⎥
⎢                         ⎥
⎣0  0  0  0  0  0  0  0  1⎦

$$ B = \begin{bmatrix} b_1 \\ b_2 \\ b_3 \\ b_4 \\ b_5 \\ b_6 \\ b_7 \\ b_8 \\ b_9 \end{bmatrix} = \begin{bmatrix} 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\ 0 &0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\ 0 &0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\ 0 &0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\ 0 &0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\ 0 &0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\ 0 &0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \end{bmatrix} = I$$

# Change of Basis

_Question_: **Is there another basis, which is a linear combination of orthogonal basis, that best re-express the data set?**

$X$ is a $m\times n$ (9 columns, 329 rows) matrix. We can do a linear transformation (rotation and stretch) on this dataset and generate a new matrix. Let us call this new matrix $Y$. In other words $P$ matrix transform $X$ to $Y$. Geometrical interpretation of $P$ is a rotation and a stretch of $X$ to obtain $Y$. The rows if the $P$ matrix is the new basis vectors that express the columns of $X$.

$$PX=Y$$

- $P$ -Linear Transformation
- $X$ -Original Data
- $Y$ -Transformed Data


 Vectors in $P$ is the **Principal Components** of $X$. There are infinite numbers of $P$ exits. In other words, we can take the original data set $X$ and rotate and stretch in any number of ways you can imagine. Among these combinations, we can pick one of them and for us to choose one transformation we can ask very fundamental questions,
 
 - What features we would like $Y$ to exhibit?
 - What is the best way to re-express the original data set $X$ ?
 - What is a good choise of basis of $P$?
 

**The best way to represent the data is to minimize the Singal to Noise ratio (_SNR_) or the ratio of variances $\sigma^2$**


$$SNR=\frac{\sigma^{2}_{signal}}{\sigma^{2}_{noise}}$$

Higher SNR ($>>1$) mean less noise and high precision data. Looking at the plot below, we can see this clearly. Less noisy the data, the sigma^2_noise is small, and the variance is much larger in the signal.


## Load Data
The data is taken from the Places Rated Almanac, by Richard Boyer and David Savageau, copyrighted and published by Rand McNally. The nine rating criteria used by Places Rated Almanac are:

- Climate & Terrain
- Housing
- Health Care & Environment
- Crime
- Transportation
- Education
- The Arts
- Recreation
- Economics

In [4]:
data = pd.read_csv('places.csv')
data = data[[
            'location',
            'climate_log10',
            'housing_log10',
            'health_log10',
            'crime_log10',
            'transportation_log10',
            'education_log10',
            'arts_log10',
            'recreation_log10',
            'economy_log10'
        ]]

data = data.rename(columns={
            'climate_log10': '$x_1$',
            'housing_log10': '$x_2$',
            'health_log10': '$x_3$',
            'crime_log10': '$x_4$',
            'transportation_log10': '$x_5$',
            'education_log10': '$x_6$',
            'arts_log10': '$x_7$',
            'recreation_log10': '$x_8$',
            'economy_log10': '$x_9$'
    })

del data['location']
data.head()

,$x_1$,$x_2$,$x_3$,$x_4$,$x_5$,$x_6$,$x_7$,$x_8$,$x_9$
0,2.716838,3.792392,2.374748,2.965202,3.605413,3.440437,2.998259,3.147676,3.882695
1,2.759668,3.910518,3.219060,2.947434,3.688687,3.387034,3.745387,3.420286,3.638489
2,2.670246,3.865637,2.790988,2.986772,3.403292,3.408240,2.374748,2.933993,3.720159
3,2.677607,3.898067,3.155640,2.785330,3.837778,3.531351,3.667920,3.208710,3.768194
4,2.818885,3.923917,3.267875,3.171141,3.816771,3.480869,3.652826,3.416973,3.757927


# Perform PCA on the dataset 

In [5]:
pca = PCA(n_components=9)
pca.fit(data)

PCA(copy=True, n_components=9, whiten=False)

# Data Analysis

**Step1**: How many Principal Components we should consider ?

Lets take a look at the eigenvalues

In [6]:
eigenDf = pd.DataFrame({
        'Component': ["PC%s" %(i) for i in range(1,10)],
        'Eigenvalue': pca.explained_variance_,
        'Proportion (%)': pca.explained_variance_ratio_.round(4) * 100
    })

eigenDf['Cumulative (%)'] = 0
cu = 0.0
for i in eigenDf.index:
    cu += eigenDf.iloc[i]['Proportion (%)']
    eigenDf.loc[i, 'Cumulative (%)'] = cu
display(eigenDf)

,Component,Eigenvalue,Proportion (%),Cumulative (%)
0,PC1,0.376315,72.27,72.27
1,PC2,0.050897,9.77,82.04
2,PC3,0.027835,5.35,87.39
3,PC4,0.022897,4.40,91.79
4,PC5,0.016720,3.21,95.00
5,PC6,0.011916,2.29,97.29
6,PC7,0.008431,1.62,98.91
7,PC8,0.003922,0.75,99.66
8,PC9,0.001792,0.34,100.00


**Eigenvalues** (variances) along each principal component show how the data variation on each component. First Principal Component show the largest change and the second component shows the second largest and so on.

**Proportion** of eigrnvalues (variation) calcuated as 

$$proportion = \frac{Eigenvalue}{\sum{(Eigenvalues)}}$$

According to the above table, the first principal component explain **72.27%** of the variance of the dataset, and second principal component describe **9.77%** of the variance of the data...etc

**Cumulative** values are calculated by adding the proportion values. The first two principal components show 82% of the variance of the data (PC1 Proportion 72.27% + PC1 Proportion 9.77%). First 3 PCs show 87.39% variance of the dataset 

**step2**: Calculate the Principal Component Scores.

First Principal Component can be calculated using First Principal Component coefficients multiply by the first data row (vector)


$$ y_1 =    (PC1[1] \times x_1) + (PC1[2] \times x_2) + (PC1[3] \times x_3) + (PC1[4] \times x_4) + (PC1[5] \times x_5) + (PC1[6] \times x_6) + (PC1[7] \times x_7) + (PC1[8] \times x_8) + (PC1[9] \times x_9)$$



$$ y_1 =  (0.03507288 \times x_1) + (0.09335159 \times x_2) + (0.40776448 \times x_3) + (0.10044536 \times x_4) + (0.15009714 \times x_5) + (0.03215319 \times x_6) + (0.87434057 \times x_7) + (0.15899622 \times x_8) + (0.01949418 \times x_9)$$


Instead of using the raw data, we can use the difference between the variables and their sample means. This is known as the translation of the random variable. This translation does not affect interpretations because the variance of the original variables are the same as those of the translated variables.

### Interpretation of the Principal Components

Compute the correlations between the original data and principal components

In [8]:
pc = zip(pca.transform(data), data)

In [50]:
df = pd.DataFrame(columns=data.columns)

for d in pc:
    df[d[1]] = d[0]
    
display(df.T)




,0,1,2,3,4,5,6,7,8
$x_1$,-0.436677,0.420163,0.118121,0.089959,0.080968,0.005028,0.077777,-0.142870,-0.002988
$x_2$,0.620958,0.005346,-0.001808,-0.100745,-0.021650,0.027288,-0.134777,0.027679,-0.067781
$x_3$,-0.873256,-0.212104,-0.049693,0.171612,-0.026654,-0.044418,0.044284,0.065949,-0.009643
$x_4$,0.502948,-0.063621,0.170495,-0.112303,0.196313,0.045515,0.030802,-0.087271,0.036426
$x_5$,0.609775,-0.007233,-0.233127,0.050234,0.071192,0.059613,-0.047273,-0.046733,0.000952
$x_6$,-0.745633,-0.188183,0.040752,0.072765,-0.063687,-0.027134,-0.040332,-0.054720,0.035484
$x_7$,0.005097,0.063496,0.378155,-0.025016,-0.095842,0.058177,0.042709,-0.015394,0.056631
$x_8$,-0.039376,-0.092385,0.095279,0.004404,0.123509,0.051707,-0.006696,-0.100406,-0.008554
$x_9$,-0.893575,-0.097862,0.107068,-0.201101,0.124011,0.170125,-0.060089,-0.042225,0.004749
